<a href="https://colab.research.google.com/github/Andres8bit/parallel-computing/blob/main/Kogge_Stone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3ddgo6am
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3ddgo6am
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=a079b64fa67a58d661c15de0665613f6f322d9b742216c1248cdca93c0778053
  Stored in directory: /tmp/pip-ephem-wheel-cache-3dsfn35s/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
# Implements basic Kogge-Stone prefix sum algorithm originally designed for use
# in paralle adders

%%cu
#include <iostream>
#define SECTION_SIZE 8
__global__ void kogge_stone_scan_kernel(float* x, float* y, int input_size);

void print_array(float *array, int width);

int main(){
    float host_input[] = {3,1,7,0,4,1,6,3};
    float host_output[SECTION_SIZE];
    float *device_input, *device_output;

    size_t mem_size = SECTION_SIZE * sizeof(float);

    cudaMalloc((void **) &device_input,mem_size);
    cudaMalloc((void **) &device_output,mem_size);

    cudaMemcpy(device_input,host_input,mem_size,cudaMemcpyHostToDevice);
    
    dim3 dim_block(32);
    dim3 dim_grid(32);

    kogge_stone_scan_kernel<<<dim_grid,dim_block>>>(device_input,device_output,SECTION_SIZE);

    cudaMemcpy(host_output,device_output,mem_size,cudaMemcpyDeviceToHost);

    printf("input:");
    print_array(host_input,SECTION_SIZE);
    
    printf("\n output:");
    print_array(host_output,SECTION_SIZE);
    
    cudaFree(device_input);
    cudaFree(device_output);
    return 0;
}

void print_array(float *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");
}
__global__ void kogge_stone_scan_kernel(float* x, float* y, int input_size){
    __shared__ float ans[SECTION_SIZE];

    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if(i < input_size)
       ans[threadIdx.x] = x[i];
    for (unsigned int stride = 1; stride < blockDim.x; stride *= 2){
        __syncthreads();
        if (threadIdx.x >= stride)
          ans[threadIdx.x] += ans[threadIdx.x - stride];
    }

    y[i] = ans[threadIdx.x];
}

input:3	1	7	0	4	1	6	3	

 output:3	4	11	11	15	16	22	25	

